# Řešení diferenciálních rovnic s okrajovou podmínkou


V předchozím cvičení jsme se zabývali metodami na řešení *počáteční úlohy* tvaru:

$$
\frac{d \vec{y}}{dx} = f(x, \vec{y}), \quad \vec{y}(0) = \vec{y}_0.
$$

Tato úloha představuje soustavu obyčejných diferenciálních rovnic 1. řádu s počáteční podmínkou $\vec{y}(0) = \vec{y}_0$. Tedy známe řešení $\vec{y}$ v nějakém konkrétním bodě.

V dnešním cvičení si představíme numerické metody řešící *okrajové úlohy*. Oproti počáteční úloze budeme znát část řešení ve více bodech. Pro $N=2$ můžeme například mít $y_0(0) = 0$ a $y_1(1) = 0$ nebo $y_0(0) = 0$ a $y_0(1) = 0$. Druhý případ je běžnější, proto se dále budeme zabývat pouze jím.

Omezíme se na řešení soustavy **dvou** ODR, která odpovídá diferenciální rovnici druhého řádu:

$$ 
y^{\prime \prime} = f(x, y(x), y^{\prime}(x)), \quad x \in [a, b], \quad y(a) = \alpha_1, \quad y(b) = \alpha_2,
$$

kde máme dvě okrajové podmínky pro funkci $y$ v bodech $a$ a $b$. Okrajová úloha již nelze řešit stejným způsobem, jako počáteční. Neznáme totiž celé jednoznačné řešení (hodnotu funkce $y$ i její derivace $y^{\prime}$) v jednom konkrétním bodě, ze kterého bychom mohli řešení vyvíjet - integrovat pravou stranu soustavy ODR.

Zde si představíme dvě metody na řešení okrajových úloh:
* Metoda střelby
* Metoda sítí (konečných diferencí)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg, integrate, optimize

## Metoda střelby

Tato metoda je inspirovaná úlohou střelby na cíl v homogenním gravitačním poli Země.

![Střelba z kanónu](../images/shootingmethod.png)

Řekněme, že máme kanón na pozici $x = a$ a cíl na pozici $x = b$. Zřejmě nás zajímá, pod jakým úhlem vystřelit a jakou silou (kolik použít střelného prachu), aby koule zasáhla cíl. Víme, že hmotný objekt se v gravitačním poli Země řídí pohybovou rovnicí (zde neuvažujeme odpor vzduchu):

$$
m \ddot{\vec{r}} = - m\vec{g}.
$$

Na začátku známe pouze polohu kanónu a cíle, což odpovídá:

$$
\vec{r}(0) = [x_0, y_0] = [a, 0], \quad \vec{r}(T) = [x_T, y_T] = [b, 0],
$$

kde $T$ značí čas dopadu koule zpátky na zem. Tato úloha je *okrajovou úlohou*, jelikož známe použe část řešení (polohy ale ne rychlosti) na začátku a na konci. Bez znalosti počátečních rychlostí (velikosti rychlosti a náměru děla) nedokážeme řešení vyvíjet integrováním pravé strany.
Metodu střelby si dále ukázeme na obecné diferenciální rovnici 2. řádu a k úloze střelby z kanónu se vrátíme v zápočtovém úkolu 5. 

Uvažujme opět následující okrajovou úlohu:

$$ 
y^{\prime \prime} = f(x, y(x), y^{\prime}(x)), \quad x \in [a, b], \quad y(a) = \alpha_1, \quad y(b) = \alpha_2,
$$

Řešení pomocí metody střelby spočívá v následujících krocích:
1. náhodně zvolíme počáteční hodnotu derivace $y^{\prime}(a) = \beta$
2. vyřešíme počáteční úlohu:

$$
\begin{align}
& y^{\prime}(x) = z(x), \quad &y(a) = \alpha_1 \\
& z^{\prime}(x) = f(x, y(x), z(x)), \quad &z(a) = y^{\prime}(a) = \beta
\end{align}
$$

3. tím získáme $y(b; \beta)$
4. řešíme nelineální rovnici $F(\beta) = y(b; \beta) - \alpha_2 = 0$ tím, že opakujeme kroky 1-3

Na krok 4 je možné aplikovat libovolnou z metod představených v [předchozím cvičení](nelin-rce). Jednoduchou volbou je *bisekce*, tedy máme na začátku dva odhady $\beta_1$ a $\beta_2$ takové, že $F(\beta_1) F(\beta_2) < 0$ (jednou přestřelíme cíl, podruhé podstřelíme). My v následujícím kódu využijeme *Newton-Rapsonovu metodu*.

In [ ]:
# vstupni matice pro reseni pocatecniho problemu - resime dve rovnice prvniho radu
def F(t,s):
    return np.dot(np.array([[0,1],[0,-9.81/s[1]]]),s)    

# prvni pocatecni podminka y(t=0)=0
y0 = 0

# odhad pocatecni rychlosti rakety
v0 = 10

# cas
t_rozsah = np.linspace(0, 5, 100)
# reseni pocatecniho problemu pomoci integrovane funkce solve_ivp() pro pocatecni podminky y0 a v0
# funkce vrati hodnoty promenne t a prislusna reseni: y = y[0], y'= y[1]
reseni = solve_ivp(F, [0, 5], [y0, v0], method='RK45', t_eval = t_rozsah)

fig, ax = plt.subplots(figsize=(10,5))
ax.plot(reseni.t, reseni.y[0],label='reseni')
ax.scatter(5, 50, marker='x',color='C1',label='cil')
ax.set_xlabel('t [s]')
ax.set_ylabel('y [m]')
ax.set_xlim(0,6)
ax.set_ylim(0,100)
ax.legend()

## Metody sítí (konečných diferencí)
